In [1]:
import sqlite3
import json
import os
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from pathlib import Path

def return_highest_frame_number(image_dir, video_name):
    """
    Return the highest frame number from image filenames in the given directory that is a multiple of 30.
    Assumes filenames are in the format: "{video_name}_{frame_number}.jpg" or "{video_name}_{frame_number}.PNG"
    """
    video_name = os.path.splitext(video_name)[0]
    video_dir = os.path.join(image_dir, video_name)
    if not os.path.exists(video_dir):
        print(f"Directory {video_dir} does not exist.")
        return -1
    max_frame = -1
    for root, _, files in os.walk(video_dir):
        for file in files:
            # Check for .jpg or .PNG
            if file.endswith(".jpg") or file.endswith(".PNG"):
                ext = ".jpg" if file.endswith(".jpg") else ".PNG"
                parts = file.rsplit("_", 1)
                if len(parts) == 2 and parts[1].endswith(ext):
                    try:
                        frame_num = int(parts[1].replace(ext, ""))
                        if frame_num % 30 == 0 and frame_num > max_frame:
                            max_frame = frame_num
                    except ValueError:
                        continue
    print(f"Highest frame number: is {max_frame}")
    return max_frame

import sqlite3
import json
import os

DB_PATH = "/home/nele_pauline_suffo/ProcessedData/quantex_annotations/annotations.db"
TARGET_CATEGORIES = (3, 4, 5, 6, 7, 8, 12)

def load_annotations_from_db(video_name=None, image_id=None):
    """
    Query the annotations table and return a list of annotation dicts.

    - Always filters: outside = 0
    - Applies object_interaction = 'Yes' only for category_id in TARGET_CATEGORIES.
      For other category_ids, object_interaction is not checked.
    - Optionally filters by video_name (resolves to video_id) and/or image_id.
    """
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()

    # Resolve video_name -> video_id if provided
    video_id = None
    if video_name is not None:
        cursor.execute("SELECT id FROM videos WHERE file_name = ?", (video_name,))
        result = cursor.fetchone()
        if result:
            video_id = result[0]
        else:
            print(f"Video name '{video_name}' not found in videos table.")
            conn.close()
            return []

    base_query = "SELECT video_id, image_id, category_id, bbox FROM annotations"
    # Always require outside = 0
    conditions = ["outside = 0"]
    params = []

    # keep rows where either:
    #  - category_id NOT IN TARGET_CATEGORIES  (no object_interaction check)
    #  OR
    #  - category_id IN TARGET_CATEGORIES AND object_interaction = 'Yes'
    cat_list_str = ", ".join(str(c) for c in TARGET_CATEGORIES)
    conditions.append(
        f"(category_id NOT IN ({cat_list_str}) OR (category_id IN ({cat_list_str}) AND object_interaction = 'Yes'))"
    )

    if video_id is not None:
        conditions.append("video_id = ?")
        params.append(video_id)
    if image_id is not None:
        conditions.append("image_id = ?")
        params.append(image_id)

    if conditions:
        query = base_query + " WHERE " + " AND ".join(conditions)
    else:
        query = base_query

    cursor.execute(query, params)
    rows = cursor.fetchall()
    conn.close()

    # Convert rows to list of dicts
    annotations = []
    for row in rows:
        video_id_val, image_id_val, category, bbox_json = row
        # bbox might be JSON string (list) or comma-separated
        try:
            bbox = json.loads(bbox_json)
        except Exception:
            bbox = [float(x) for x in bbox_json.split(",")]
        annotations.append({
            'video_id': video_id_val,
            'frame_number': image_id_val,
            'category': category,
            'bbox': bbox,
        })
    return annotations

def plot_annotations(video_name, frame_number, annotations_table, image_folder, output_folder):
    """
    Plot image and draw bounding boxes for person/face annotations (pixel-based coordinates).
    """
    # Prepare paths
    video_name = os.path.splitext(video_name)[0]
    frame_number_padded = str(frame_number).zfill(6)

    # Check for both .jpg and .PNG
    image_filename_jpg = f"{video_name}_{frame_number_padded}.jpg"
    image_filename_png = f"{video_name}_{frame_number_padded}.PNG"
    image_path_jpg = os.path.join(image_folder, video_name, image_filename_jpg)
    image_path_png = os.path.join(image_folder, video_name, image_filename_png)

    # Determine which image exists
    if os.path.exists(image_path_jpg):
        image_path = image_path_jpg
        output_filename = image_filename_jpg
    elif os.path.exists(image_path_png):
        image_path = image_path_png
        output_filename = image_filename_png
    else:
        print(f"⚠️ Image not found for frame {frame_number_padded}: neither {image_filename_jpg} nor {image_filename_png}")
        return

    # Open image
    img = Image.open(image_path)
    img_w, img_h = img.size
 
    # Create figure with same aspect ratio as image
    fig, ax = plt.subplots(1, figsize=(img_w / 100, img_h / 100), dpi=100)
    ax.imshow(img)
    ax.axis('off')

    # Draw bounding boxes
    for ann in annotations_table:
        bbox = [float(x) for x in ann['bbox']]

        # Detect format automatically
        if bbox[2] > 1.0 and bbox[3] > 1.0 and (bbox[2] > bbox[0] and bbox[3] > bbox[1]):
            # likely [x_min, y_min, x_max, y_max]
            x, y, w, h = bbox[0], bbox[1], bbox[2] - bbox[0], bbox[3] - bbox[1]
        else:
            # likely [x, y, w, h]
            x, y, w, h = bbox

        # Clamp to image bounds
        x = max(0, min(x, img_w - 1))
        y = max(0, min(y, img_h - 1))
        w = min(w, img_w - x)
        h = min(h, img_h - y)

        rect = patches.Rectangle(
            (x, y),
            w, h,
            linewidth=2,
            edgecolor='b',
            facecolor='none'
        )
        ax.add_patch(rect)
        ax.text(x, y - 5, str(ann['category']),
                color='white', fontsize=8, backgroundcolor='black')

    # Save output
    Path(output_folder).mkdir(parents=True, exist_ok=True)
    output_path = os.path.join(output_folder, output_filename)
    plt.savefig(output_path, bbox_inches='tight', pad_inches=0)
    plt.close(fig)
    print(f"✅ Saved annotated image to {output_path}")

In [29]:
video_name = "quantex_at_home_id255237_2022_05_08_04.mp4"
output_dir = "/home/nele_pauline_suffo/ProcessedData/frame_validation"
image_dir = "/home/nele_pauline_suffo/ProcessedData/quantex_rawframes_cvat"
image_dir = "/home/nele_pauline_suffo/ProcessedData/quantex_videos_processed"
frame = return_highest_frame_number(image_dir, video_name)
#frame = 11430
annotations = load_annotations_from_db(video_name, frame)
plot_annotations(video_name, frame, annotations, image_dir, output_dir)

Highest frame number: is 13230


✅ Saved annotated image to /home/nele_pauline_suffo/ProcessedData/frame_validation/quantex_at_home_id255237_2022_05_08_04_013230.jpg


In [28]:
annotations

[]

In [ ]:
annotations

[{'video_id': 24,
  'frame_number': 53940,
  'category': 5,
  'bbox': [2019.46, 57.35, 2304.0, 362.84],
  'outside': 0},
 {'video_id': 24,
  'frame_number': 53940,
  'category': 12,
  'bbox': [1492.87, 0.0, 2304.0, 646.29],
  'outside': 0},
 {'video_id': 24,
  'frame_number': 53940,
  'category': 11,
  'bbox': [0.0, 1005.51, 276.8, 1296.0],
  'outside': 0}]